In [3]:
%%writefile static.cu

#include <stdio.h>

#define N 10

//Define a static_global memory array

__device__ int globalArray[N];

// Kernel function to modify the static global memory
__global__ void incrementGlobalArray(){
  int idx = threadIdx.x;

// Each thread increments its corresponding index in the global array
  if (idx < N){
    globalArray[idx] += 5;
  }

}


// Host function to initialize the static global memory from the host
void initializeGlobalArray(int *hostArray){

  cudaMemcpyToSymbol(globalArray, hostArray, N*sizeof(int), 0, cudaMemcpyHostToDevice);

}

// Host function to retrive the static global memory from the device
void retrieveGlobalArray(int *hostArray){

  cudaMemcpyFromSymbol(hostArray, globalArray, N*sizeof(int), 0, cudaMemcpyDeviceToHost);

}



int main()
{

  int hostArray[N];

  // Initialize the host array

  for (int i=0; i<N; i++)
  {
    hostArray[i]=i;
  }

  // Initialize globalArray on the device with values from hostArray
  initializeGlobalArray(hostArray);

  //Launch kernel with N threads (one per element in the array)
  incrementGlobalArray<<<1,N>>>();

  //Retrieve modified globalArray from the device
  retrieveGlobalArray(hostArray);

  //Print the modeified global array
  printf("Modified globalArray: \n");
  for (int i=0; i<N; i++)
  {
    printf("%d ",hostArray[i]);
  }

  printf("\n");

  return 0;
}


Overwriting static.cu


In [4]:
!nvcc -o exe static.cu
!./exe

Modified globalArray: 
5 6 7 8 9 10 11 12 13 14 
